In [1]:
import pandas as pd
import numpy as np
import itertools
from itertools import chain, combinations
import statsmodels.formula.api as smf
import scipy.stats as scipystats
import statsmodels.api as sm
import statsmodels.stats.stattools as stools
import statsmodels.stats as stats
from statsmodels.graphics.regressionplots import *
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from sklearn.cross_validation import train_test_split
import math
import time

/Users/naveenkumar/anaconda/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
elemapi2 = pd.read_csv('/Users/naveenkumar/Dropbox/OnlineWork/elemapi2.csv')
elemapi2.columns

Index([u'snum', u'dnum', u'api00', u'api99', u'growth', u'meals', u'ell',
       u'yr_rnd', u'mobility', u'acs_k3', u'acs_46', u'not_hsg', u'hsg',
       u'some_col', u'col_grad', u'grad_sch', u'avg_ed', u'full', u'emer',
       u'enroll', u'mealcat', u'collcat'],
      dtype='object')

In [3]:
elemapi2_sel = elemapi2[["api00", "some_col", "yr_rnd", "mealcat"]]
print elemapi2_sel.describe()

def cv_desc(df, var):
    return df[var].value_counts(dropna = False)

print '\n'  
print cv_desc(elemapi2_sel, 'mealcat')
print '\n'  
print cv_desc(elemapi2_sel, 'yr_rnd')

            api00    some_col     yr_rnd     mealcat
count  400.000000  400.000000  400.00000  400.000000
mean   647.622500   19.712500    0.23000    2.015000
std    142.248961   11.336938    0.42136    0.819423
min    369.000000    0.000000    0.00000    1.000000
25%    523.750000   12.000000    0.00000    1.000000
50%    643.000000   19.000000    0.00000    2.000000
75%    762.250000   28.000000    0.00000    3.000000
max    940.000000   67.000000    1.00000    3.000000


3.0    137
2.0    132
1.0    131
Name: mealcat, dtype: int64


0.0    308
1.0     92
Name: yr_rnd, dtype: int64


In [4]:
def codebook(df, var):
    title = "Codebook for " + str(var)
    unique_values = len(df[var].unique())
    max_v = df[var].max()
    min_v = df[var].min()
    n_miss = sum(pd.isnull(df[var]))
    mean = df[var].mean()
    stdev = df[var].std()
    print pd.DataFrame({'title': title, 'unique values': unique_values, 'max value' : max_v, 'min value': min_v, 'num of missing' : n_miss, 'mean' : mean, 'stdev' : stdev}, index = [0])
    return

codebook(elemapi2_sel, 'api00')

   max value      mean  min value  num of missing       stdev  \
0      940.0  647.6225      369.0               0  142.248961   

                title  unique values  
0  Codebook for api00            271  


In [5]:
reg = smf.ols(formula = "api00 ~ yr_rnd", data = elemapi2_sel).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  api00   R-squared:                       0.226
Model:                            OLS   Adj. R-squared:                  0.224
Method:                 Least Squares   F-statistic:                     116.2
Date:                Sun, 14 Jan 2018   Prob (F-statistic):           5.96e-24
Time:                        22:39:49   Log-Likelihood:                -2498.9
No. Observations:                 400   AIC:                             5002.
Df Residuals:                     398   BIC:                             5010.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    684.5390      7.140     95.878      0.000     670.503     698.575
yr_rnd      -160.5064     14.887    -10.782      0.000    -189.774    -131.239
==============================================================================
Omnibus:                       45.748   Durbin-Watson:                   1.499
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               13.162
Skew:                           0.006   Prob(JB):                      0.00139
Kurtosis:                       2.111   Cond. No.                         2.53
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [6]:
elemapi2_sel['yr_rnd'].value_counts()
elemapi2_sel['yr_rnd_c'] = elemapi2_sel['yr_rnd'].map({0: "No", 1: "Yes"})
elemapi2_sel['yr_rnd_c'].value_counts()

/Users/naveenkumar/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


No     308
Yes     92
Name: yr_rnd_c, dtype: int64

In [7]:
elemapi2_sel['yr_rnd_c'] = elemapi2_sel['yr_rnd'].map({0: "No", 1: "Yes"})
elemapi2_sel['mealcat_c'] = elemapi2_sel['mealcat'].map({1: "0-46% free meals", 2: "47-80% free meals", 3: "1-100% free meals"})

elemapi2_sel_group = elemapi2_sel.groupby('yr_rnd_c')
print(elemapi2_sel_group['api00'].agg([np.mean, np.std]))


elemapi2_sel_group = elemapi2_sel.groupby('mealcat_c')
print(elemapi2_sel_group['api00'].agg([lambda x: x.shape[0], np.mean, np.std]))

/Users/naveenkumar/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/naveenkumar/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


                mean         std
yr_rnd_c                        
No        684.538961  132.112534
Yes       524.032609   98.916043
                   <lambda>        mean        std
mealcat_c                                         
0-46% free meals      131.0  805.717557  65.668664
1-100% free meals     137.0  504.379562  62.727015
47-80% free meals     132.0  639.393939  82.135130


In [8]:
lm = smf.ols('api00 ~ mealcat_c', data = elemapi2_sel).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  api00   R-squared:                       0.755
Model:                            OLS   Adj. R-squared:                  0.754
Method:                 Least Squares   F-statistic:                     611.1
Date:                Sun, 14 Jan 2018   Prob (F-statistic):          6.48e-122
Time:                        22:39:49   Log-Likelihood:                -2269.0
No. Observations:                 400   AIC:                             4544.
Df Residuals:                     397   BIC:                             4556.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                        805.7176      6.169    130.599      0.000     793.589     817.846
mealcat_c[T.1-100% free meals]  -301.3380      8.629    -34.922      0.000    -318.302    -284.374
mealcat_c[T.47-80% free meals]  -166.3236      8.708    -19.099      0.000    -183.444    -149.203
==============================================================================
Omnibus:                        1.593   Durbin-Watson:                   1.541
Prob(Omnibus):                  0.451   Jarque-Bera (JB):                1.684
Skew:                          -0.139   Prob(JB):                        0.431
Kurtosis:                       2.847   Cond. No.                         3.76
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
from sklearn import preprocessing
le_mealcat = preprocessing.LabelEncoder()
elemapi2_sel['mealcat_dummy'] = le_mealcat.fit_transform(elemapi2_sel.mealcat)
elemapi2_sel.groupby('mealcat_dummy').size()

/Users/naveenkumar/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


mealcat_dummy
0    131
1    132
2    137
dtype: int64

In [11]:
ohe = preprocessing.OneHotEncoder()
dummy = pd.DataFrame(ohe.fit_transform(elemapi2_sel['mealcat'].reshape(-1,1)).toarray() , columns = ["mealcat1", "mealcat2", "mealcat3"])
elemapi2_sel = pd.concat([elemapi2_sel, dummy], axis = 1)
elemapi2_sel.head()

/Users/naveenkumar/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


,api00,some_col,yr_rnd,mealcat,yr_rnd_c,mealcat_c,mealcat_dummy,mealcat1,mealcat2,mealcat3
0,693.0,0.0,0.0,2.0,No,47-80% free meals,1,0.0,1.0,0.0
1,570.0,0.0,0.0,3.0,No,1-100% free meals,2,0.0,0.0,1.0
2,546.0,0.0,0.0,3.0,No,1-100% free meals,2,0.0,0.0,1.0
3,571.0,9.0,0.0,3.0,No,1-100% free meals,2,0.0,0.0,1.0
4,478.0,0.0,0.0,3.0,No,1-100% free meals,2,0.0,0.0,1.0


In [12]:
lm = smf.ols('api00 ~ mealcat2 + mealcat3', data = elemapi2_sel).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  api00   R-squared:                       0.755
Model:                            OLS   Adj. R-squared:                  0.754
Method:                 Least Squares   F-statistic:                     611.1
Date:                Sun, 14 Jan 2018   Prob (F-statistic):          6.48e-122
Time:                        22:41:31   Log-Likelihood:                -2269.0
No. Observations:                 400   AIC:                             4544.
Df Residuals:                     397   BIC:                             4556.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    805.7176      6.169    130.599      0.000     793.589     817.846
mealcat2    -166.3236      8.708    -19.099      0.000    -183.444    -149.203
mealcat3    -301.3380      8.629    -34.922      0.000    -318.302    -284.374
==============================================================================
Omnibus:                        1.593   Durbin-Watson:                   1.541
Prob(Omnibus):                  0.451   Jarque-Bera (JB):                1.684
Skew:                          -0.139   Prob(JB):                        0.431
Kurtosis:                       2.847   Cond. No.                         3.76
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
lm = lm = smf.ols('api00 ~ C(mealcat)', data = elemapi2_sel).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  api00   R-squared:                       0.755
Model:                            OLS   Adj. R-squared:                  0.754
Method:                 Least Squares   F-statistic:                     611.1
Date:                Sun, 14 Jan 2018   Prob (F-statistic):          6.48e-122
Time:                        22:46:05   Log-Likelihood:                -2269.0
No. Observations:                 400   AIC:                             4544.
Df Residuals:                     397   BIC:                             4556.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept           805.7176      6.169    130.599      0.000     793.589     817.846
C(mealcat)[T.2.0]  -166.3236      8.708    -19.099      0.000    -183.444    -149.203
C(mealcat)[T.3.0]  -301.3380      8.629    -34.922      0.000    -318.302    -284.374
==============================================================================
Omnibus:                        1.593   Durbin-Watson:                   1.541
Prob(Omnibus):                  0.451   Jarque-Bera (JB):                1.684
Skew:                          -0.139   Prob(JB):                        0.431
Kurtosis:                       2.847   Cond. No.                         3.76
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
elemapi2_sel = elemapi2.ix[:, ["api00", "some_col", "yr_rnd", "mealcat"]]

ohe = preprocessing.OneHotEncoder()
dummy = pd.DataFrame(ohe.fit_transform(elemapi2_sel.mealcat.reshape(-1,1)).toarray(), columns = ["mealcat1", "mealcat2", "mealcat3"])
mealxynd = np.multiply(dummy, elemapi2.yr_rnd.reshape(-1, 1))
mealxynd.columns = ["mealxynd1", "mealxynd2", "mealxynd3"]
elemapi2_sel = pd.concat([elemapi2_sel, dummy, mealxynd], axis = 1)

elemapi2_sel.groupby(["yr_rnd", "mealcat", "mealxynd1", "mealxynd2", "mealxynd3"]).size()

/Users/naveenkumar/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.
/Users/naveenkumar/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  after removing the cwd from sys.path.
/Users/naveenkumar/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """


yr_rnd  mealcat  mealxynd1  mealxynd2  mealxynd3
0.0     1.0      0.0        0.0        0.0          124
        2.0      0.0        0.0        0.0          117
        3.0      0.0        0.0        0.0           67
1.0     1.0      1.0        0.0        0.0            7
        2.0      0.0        1.0        0.0           15
        3.0      0.0        0.0        1.0           70
dtype: int64